## Compute Table

In [65]:
#BUILD_DIR = '../remote-data/build/benchmarks/'
BUILD_DIR = '../build/benchmarks/'
BASELINE = 'pgo-full'
REG0_PROFILE = ['profile dynamic push', 'profile dynamic pop', 'profile dynamic spill', 'profile dynamic reload', 'profile static push', 'profile static pop', 'profile static spill', 'profile static reload']
REG1_PROFILE = ['all push', 'all pop', 'in_unlikely push', 'in_unlikely pop', 'in_hot push', 'in_hot pop', 'in_split push', 'in_split pop', 'in_startup push', 'in_startup pop']
REG2_PROFILE = ['dynamorio push', 'dynamorio pop']
REG3_PROFILE = ['dynamic push', 'dynamic pop', 'dynamic spill', 'dynamic reload']
PERF_PROFILE = ['instructions', 'cycles', 'L1-icache-miss', 'L1-dcache-loads', 'L1-dcache-stores', 'time elapsed']
VARANT = ['none'] + ["{}-{}".format(i, j) for i in [1,3,5,10] for j in [10, 20]]
FDOIPRA = [b + 'fdoipra' + (str(i) if i != 1 else "") for i in range(1, 7) for b in ["", "b"]] 
METHODS = [BASELINE] + ['pgo-full-'+v for v in ['ipra']+FDOIPRA]

In [66]:
import os
from IPython.display import HTML, display

def load_regprof0(content):
    assert(len(content) <= 8)
    result = {}
    for line in content:
        if line.startswith('dynamic push: '):
            result['profile dynamic push'] = int(line[14:])
        if line.startswith('dynamic pop: '):
            result['profile dynamic pop'] = int(line[13:])
        if line.startswith('dynamic spill: '):
            result['profile dynamic spill'] = int(line[15:])
        if line.startswith('dynamic reload: '):
            result['profile dynamic reload'] = int(line[16:])
        if line.startswith('static push: '):
            result['profile static push'] = int(line[13:])
        if line.startswith('static pop: '):
            result['profile static pop'] = int(line[12:])
        if line.startswith('static spill: '):
            result['profile static spill'] = int(line[14:])
        if line.startswith('static reload: '):
            result['profile static reload'] = int(line[15:])
    return result

def load_regprof1(content):
    assert(len(content) <= 10)
    result = {}
    for line in content:
        if line.startswith('all push: '):
            result['all push'] = int(line[10:])
        if line.startswith('all pop: '):
            result['all pop'] = int(line[10:])
        if line.startswith('in_unlikely push: '):
            result['in_unlikely push'] = int(line[18:])
        if line.startswith('in_unlikely pop: '):
            result['in_unlikely pop'] = int(line[18:])
        if line.startswith('in_hot push: '):
            result['in_hot push'] = int(line[13:])
        if line.startswith('in_hot pop: '):
            result['in_hot pop'] = int(line[13:])
        if line.startswith('in_split push: '):
            result['in_split push'] = int(line[15:])
        if line.startswith('in_split pop: '):
            result['in_split pop'] = int(line[15:])
        if line.startswith('in_startup push: '):
            result['in_startup push'] = int(line[17:])
        if line.startswith('in_startup pop: '):
            result['in_startup pop'] = int(line[17:])
    return result

def load_regprof2(content):
    assert(len(content) <= 2)
    result = {}
    for line in content:
        if line.startswith('dynamic push: '):
            result['dynamorio push'] = int(line[14:])
        if line.startswith('dynamic pop: '):
            result['dynamorio pop'] = int(line[13:])
    return result

def load_regprof3(content):
    assert(len(content) <= 4)
    result = {}
    for line in content:
        if line.startswith('dynamic push: '):
            result['dynamic push'] = int(line[14:])
        if line.startswith('dynamic pop: '):
            result['dynamic pop'] = int(line[13:])
        if line.startswith('dynamic spill: '):
            result['dynamic spill'] = int(line[15:])
        if line.startswith('dynamic reload: '):
            result['dynamic reload'] = int(line[16:])
    return result

def load_bench(content):
    def get(line, key):
        p = line.find(key)
        if p != -1:
            q = line.find('( +-')
            qe = line[q:].find(')')
            return True, line[:p].strip(), line[q+5:qe-1]
        return False, None, None
    result = {}
    for line in content:
        for c in PERF_PROFILE:
            found, data, error = get(line, c)
            if found:
                if c == 'time elapsed':
                    data = data.split()[0]
                result[c] = (float(data), error)
    return result

def get_index(key):
    index = None
    if key in REG0_PROFILE:
        index = 'regprof0'
    if key in REG1_PROFILE:
        index = 'regprof1'
    if key in REG2_PROFILE:
        index = 'regprof2'
    if key in REG3_PROFILE:
        index = 'regprof3'
    if key in PERF_PROFILE:
        index = 'bench'
    return index

class Benchmark:
    def __init__(self, name):
        self.name = name
        self.results = {}
        self.best_reg = {'bench':{}, 'regprof0':{}, 'regprof1':{}, 'regprof2':{}, 'regprof3':{}}
        self.best_bench = {}
    
    def load(self, path, cases=['bench', 'regprof0', 'regprof1', 'regprof2', 'regprof3']):
        self.field = cases
        filter = {
            'bench': load_bench,
            'regprof0': load_regprof0,
            'regprof1': load_regprof1,
            'regprof2': load_regprof2,
            'regprof3': load_regprof3
        }
        for case in cases:
            result = {}
            files = [path+'/'+f for f in os.listdir(path) 
                        if os.path.isfile(path+'/'+f)]
            for file in files:
                if file.endswith('.'+case):
                    with open(file, 'r') as f:
                        _ , name = os.path.split(file)
                        result[name[:-len(case)-1]] = filter[case](f.readlines())
            if len(result) > 0:
                self.results[case] = result
        return len(self.results) != 0
    
    def get(self, key, method, var):
        def get_value(value):
            data, error = None, None
            if type(value) is tuple:
                data, error = value
            else:
                data = value
            return data, error
        try:
            index = get_index(key)
            if var == 'none':
                return get_value(self.results[index][method][key])
            return get_value(self.results[index][method+'.'+var][key])
        except:
            return None, None

    def better(self, key, method, var):
        base, _ = self.get(key, BASELINE, 'none')
        impl, _ = self.get(key, method, var)
        if base != None and base != 0 and impl != None:
            return (impl - base) * 100 / base
        else:
            return None

    def best(self, key, method):
        def compute_best(profile):
            data = []
            for var in VARANT:
                improve = 0
                detail = {}
                for k in profile:
                    if self.better(k, method, var) != None:
                        improve += self.better(k, method, var)
                        detail[k], _ = self.get(k, method, var)
                if len(detail) > 0:
                    data.append((improve, detail))
            if len(data) == 0:
                return {}
            _, result = min(data)
            return result
        def query(profile):
            if key in profile:
                index = get_index(key)
                if method in self.best_reg[index] and key in self.best_reg[index][method]:
                    return self.best_reg[index][method][key]
                self.best_reg[index][method] = compute_best(profile)
                if method in self.best_reg[index] and key in self.best_reg[index][method]:
                    return self.best_reg[index][method][key]
            return None
        for profile in [PERF_PROFILE, REG0_PROFILE, REG1_PROFILE, REG2_PROFILE, REG3_PROFILE]:
            d = query(profile)
            if d != None:
                return d
        return None

    def __str__(self) -> str:
        s = self.name + ': \n'
        for case, data in self.results.items():
            s += case
            for j in data.keys():
                s += ("    " + j)
            s += '\n'
        return s

def load_build_folder(build_folder = '../build/benchmarks/'):
    """Load a build folder and return a list of results"""
    benchmarks = []
    build_folder = os.path.abspath(build_folder)
    print(build_folder)
    folders = [f for f in os.listdir(build_folder) 
                        if os.path.isdir(f)]    
    for folder in folders:
        B = Benchmark(folder)
        if B.load(build_folder+'/'+folder):
            benchmarks.append(B)
    return benchmarks

def printTable(data, header=None, title=None, rowspan=0):
    def gen_row(row, el='td'):
        gen_row.counter += 1
        if gen_row.counter == rowspan:
            gen_row.counter = 0
        tmp = '<'+el+'{}>{}</'+el+'>' 
        tmp2 = '</'+el+'><'+el+'>' 
        return tmp.format(
            ' rowspan="'+str(rowspan)+'"' if gen_row.counter==1 else "", 
            tmp2.join(str(_) for _ in row))
    gen_row.counter = 0 if header == None else -1
    display(HTML('<table><h2>{}</h2><tr>{}</tr></table>'.format(title,
        (gen_row(header, 'th') + '</tr><tr>' if header else '') + 
            '</tr><tr>'.join(gen_row(row) for row in data))
    ))

Benchmarks = load_build_folder(BUILD_DIR)

# for bench in Benchmarks:
#     print(bench)



/home/riple/IPRA-exp/build/benchmarks


In [67]:
color_palette = ['#388e3c', '#8bc34a', '#cae774', '#fff59d', '#ffbe5b', '#ed6d54', '#dd4747' ]
color_range = [-100, -3, -1, -0.1, 0.1, 1, 3, 10000]
def gen_color(better):
    for i in range(0, 7):
        if color_range[i] < better and better <= color_range[i+1]:
            return color_palette[i]
def gen_span(cell, better):
    return "{} (<span style='color: {};'>{:+.4}%</span>)".format(
            cell,
            gen_color(better),
            better)

def show_regprof(title, Bench: Benchmark, actions):
    data = []
    header = VARANT 
    for case in FDOIPRA:
        for action in actions:
            row = []
            if action == actions[0]:
                row.append(case)
            row.append(action)
            for var in header:
                better = Bench.better(action, 'pgo-full-'+case, var)
                if better == None:
                    row.append('')
                    continue
                cell, _ = Bench.get(action, 'pgo-full-'+case, var)
                row.append(gen_span(cell, better))
            cell = Bench.best(action, 'pgo-full-'+case)
            base, _ = Bench.get(action, BASELINE, 'none')
            if cell==None or base == None:
                row.append('')
            else:
                better = (cell - base) * 100 / base
                row.append(gen_span(cell, better))
            data.append(row)
    printTable(data, ['', ''] + header + ['best'], title + " - " + Bench.name, len(actions))

def show_regprof_summary(title, actions):
    data = []
    header = ['',''] + [Bench.name for Bench in Benchmarks]
    for case in METHODS:
        for action in actions:
            row = []
            if action == actions[0]:
                row.append(case)
            row.append(action)
            for Bench in Benchmarks:
                cell = Bench.best(action, case)
                base, _ = Bench.get(action, BASELINE, 'none')
                if cell==None or base == None:
                    row.append('')
                    continue
                better = (cell - base) * 100 / base
                row.append(gen_span(cell, better))
            data.append(row)
    printTable(data, header, title, len(actions))

## Push/Pop and Spill/Reload

In [68]:
show_regprof_summary("Sampled Push and Push Counting", ['all push', 'all pop'])

for bench in Benchmarks:
    show_regprof("Sampled Push and Push Counting", bench, ['all push', 'all pop'])

In [69]:
show_regprof_summary("Push and Push Counting", ['dynamic push', 'dynamic pop'])

for bench in Benchmarks:
    show_regprof("Push and Push Counting", bench, ['dynamic push', 'dynamic pop'])

In [70]:
show_regprof_summary("Spill and Reload Bytes", ['dynamic spill', 'dynamic reload'])

for bench in Benchmarks:
    show_regprof("Spill and Reload Bytes", bench, ['dynamic spill', 'dynamic reload'])

## Wall Time and Cache Loads/Stores



In [62]:
show_regprof_summary("Wall Time", ['time elapsed', 'instructions', 'cycles'])

for bench in Benchmarks:
    show_regprof("Wall Time", bench, ['time elapsed', 'instructions', 'cycles'])

In [63]:
show_regprof_summary("Cache Loads/Stores", ['L1-dcache-loads', 'L1-dcache-stores'])

for bench in Benchmarks:
    show_regprof("Cache Loads/Stores", bench, ['L1-dcache-loads', 'L1-dcache-stores'])

In [64]:
show_regprof_summary("ICache Miss Rate", ['L1-icache-miss'])

for bench in Benchmarks:
    show_regprof("ICache Miss Rate", bench, ['L1-icache-miss'])

,,clang,mysql,gcc,leveldb
pgo-full,L1-icache-miss,10122465204.0 (+0.0%),21382550245.0 (+0.0%),6600784923.0 (+0.0%),2031075144.0 (+0.0%)
pgo-full-ipra,L1-icache-miss,10179989839.0 (+0.5683%),21282138808.0 (-0.4696%),6558670689.0 (-0.638%),2005940188.0 (-1.238%)
pgo-full-fdoipra,L1-icache-miss,,,,
pgo-full-bfdoipra,L1-icache-miss,,,,
pgo-full-fdoipra2,L1-icache-miss,10210584333.0 (+0.8705%),,,
pgo-full-bfdoipra2,L1-icache-miss,,21480052057.0 (+0.456%),,2008370319.0 (-1.118%)
pgo-full-fdoipra3,L1-icache-miss,,,,
pgo-full-bfdoipra3,L1-icache-miss,10039546447.0 (-0.8192%),,,
pgo-full-fdoipra4,L1-icache-miss,,,,
pgo-full-bfdoipra4,L1-icache-miss,,,,


,,none,1-10,1-20,3-10,3-20,5-10,5-20,10-10,10-20,best
fdoipra,L1-icache-miss,,,,,,,,,,
bfdoipra,L1-icache-miss,,,,,,,,,,
fdoipra2,L1-icache-miss,,,,,,,,10210584333.0 (+0.8705%),,10210584333.0 (+0.8705%)
bfdoipra2,L1-icache-miss,,,,,,,,,,
fdoipra3,L1-icache-miss,,,,,,,,,,
bfdoipra3,L1-icache-miss,10039546447.0 (-0.8192%),,,,,,,,,10039546447.0 (-0.8192%)
fdoipra4,L1-icache-miss,,,,,,,,,,
bfdoipra4,L1-icache-miss,,,,,,,,,,
fdoipra5,L1-icache-miss,,,,,,,,,,
bfdoipra5,L1-icache-miss,,,,,,,,,,


,,none,1-10,1-20,3-10,3-20,5-10,5-20,10-10,10-20,best
fdoipra,L1-icache-miss,,,,,,,,,,
bfdoipra,L1-icache-miss,,,,,,,,,,
fdoipra2,L1-icache-miss,,,,,,,,,,
bfdoipra2,L1-icache-miss,,,21480052057.0 (+0.456%),,,,,,,21480052057.0 (+0.456%)
fdoipra3,L1-icache-miss,,,,,,,,,,
bfdoipra3,L1-icache-miss,,,,,,,,,,
fdoipra4,L1-icache-miss,,,,,,,,,,
bfdoipra4,L1-icache-miss,,,,,,,,,,
fdoipra5,L1-icache-miss,,,,,,,,,,
bfdoipra5,L1-icache-miss,,,,,,,,,,


,,none,1-10,1-20,3-10,3-20,5-10,5-20,10-10,10-20,best
fdoipra,L1-icache-miss,,,,,,,,,,
bfdoipra,L1-icache-miss,,,,,,,,,,
fdoipra2,L1-icache-miss,,,,,,,,,,
bfdoipra2,L1-icache-miss,,,,,,,,,,
fdoipra3,L1-icache-miss,,,,,,,,,,
bfdoipra3,L1-icache-miss,,,,,,,,,,
fdoipra4,L1-icache-miss,,,,,,,,,,
bfdoipra4,L1-icache-miss,,,,,,,,,,
fdoipra5,L1-icache-miss,,,,,,,,,,
bfdoipra5,L1-icache-miss,,,,,,,,,,


,,none,1-10,1-20,3-10,3-20,5-10,5-20,10-10,10-20,best
fdoipra,L1-icache-miss,,,,,,,,,,
bfdoipra,L1-icache-miss,,,,,,,,,,
fdoipra2,L1-icache-miss,,,,,,,,,,
bfdoipra2,L1-icache-miss,,,2008370319.0 (-1.118%),,,,,,,2008370319.0 (-1.118%)
fdoipra3,L1-icache-miss,,,,,,,,,,
bfdoipra3,L1-icache-miss,,,,,,,,,,
fdoipra4,L1-icache-miss,,,,,,,,,,
bfdoipra4,L1-icache-miss,,,,,,,,,,
fdoipra5,L1-icache-miss,,,,,,,,,,
bfdoipra5,L1-icache-miss,,,,,,,,,,
